## PyTorch Resnets

In [24]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
project_dir = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..', '..', '..')))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_STATIC_PATH
)
from experiments.utils.loader import Loader

In [25]:
series = 69
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = 'resnet-human'
series_path = os.path.join(
    NODE_PROFILING_RESULTS_STATIC_PATH,
    'series',
    str(series))
loader = Loader(
    series_path=series_path, config_key_mapper=config_key_mapper,
    model_name=model_name)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
# print(results.columns)
# print(key_config_df.columns)
# results

In [26]:
results.columns

Index(['experiment_id', 'client_to_model_latencies_avg',
       'client_to_model_latencies_p99', 'client_to_model_latencies_p50',
       'client_to_model_latencies_var', 'client_to_model_latencies_max',
       'client_to_model_latencies_min', 'model_latencies_avg',
       'model_latencies_p99', 'model_latencies_p50', 'model_latencies_var',
       'model_latencies_max', 'model_latencies_min',
       'model_to_client_latencies_avg', 'model_to_client_latencies_p99',
       'model_to_client_latencies_p50', 'model_to_client_latencies_var',
       'model_to_client_latencies_max', 'model_to_client_latencies_min',
       'start_time', 'end_time', 'duration', 'timeout_count',
       'cpu_usage_count_avg', 'cpu_usage_count_p99', 'cpu_usage_count_p50',
       'cpu_usage_count_var', 'cpu_usage_count_max', 'cpu_usage_count_min',
       'cpu_usage_rate_avg', 'cpu_usage_rate_p99', 'cpu_usage_rate_p50',
       'cpu_usage_rate_var', 'cpu_usage_rate_max', 'cpu_usage_rate_min',
       'cpu_throttled_coun

In [27]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config) 

File name: 0.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1', '2', '4', '8'],
    'data_type': 'image',
    'max_batch_size': ['1', '2', '8', '32', '64'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': [   'resnet18',
                          'resnet34',
                          'resnet50',
                          'resnet101',
                          'resnet152'],
    'node_name': 'resnet-human',
    'num_interop_threads': ['1'],
    'num_threads': ['1'],
    'pipeline_name': 'video',
    'repetition': 1,
    'replicas': [1],
    'series': 69,
    'series_meta': 'all variables change for making the profiler 1,\\n cpu '
                   'type: Intel(R) Xeon(R) Gold 6126 CPU @ 2.60GHz',
    'timeout': 1,
    'use_threading': 'True',
    'workload_config': {'load_duration': 30, 'loads_to_test': [1, 10, 20]},
    'workload_type': 'static'}
File name: 1.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1',

In [28]:
display(key_config_df)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas,no_engine,mode,data_type,benchmark_duration
0,1,video,resnet-human,resnet18,1,10Gi,1,1,1,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1
1,2,video,resnet-human,resnet18,1,10Gi,1,1,10,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1
2,3,video,resnet-human,resnet18,1,10Gi,1,1,20,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1
3,4,video,resnet-human,resnet18,2,10Gi,1,1,1,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1
4,5,video,resnet-human,resnet18,2,10Gi,1,1,10,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,video,resnet-human,resnet152,4,10Gi,64,1,10,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1
296,297,video,resnet-human,resnet152,4,10Gi,64,1,20,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1
297,298,video,resnet-human,resnet152,8,10Gi,64,1,1,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1
298,299,video,resnet-human,resnet152,8,10Gi,64,1,10,30,69,all variables change for making the profiler 1...,1,False,exponential,image,1


In [29]:
experiment_ids = key_config_df[
    (key_config_df['model_variant'] == 'resnet18') & (key_config_df['cpu_request'] == 1)]['experiment_id'].tolist()
metadata_columns = ['model_variant', 'max_batch_size', 'cpu_request', 'load']
results_columns = ['model_latencies_min', 'model_latencies_p99', 'cpu_usage_count_avg', 'model_latencies_avg', 'throughput_avg']
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns)
# output['throughput'] = 1/output['model_latencies_avg'] * output['max_batch_size']
# output.sort_values(by='throughput')
display(output)
# ax = output.plot.bar(x='max_batch_size', y=['throughput'])
# ax.set_xlabel("Max Batch Size")
# ax.set_ylabel("Throughput (RPS)")

,model_variant,max_batch_size,cpu_request,load,model_latencies_min,model_latencies_p99,cpu_usage_count_avg,model_latencies_avg,throughput_avg
0,resnet18,1,1,1,0.048396,0.126438,8.446636,0.080438,0.129234
1,resnet18,1,1,10,0.043589,0.079451,5.072055,0.059833,1.273083
2,resnet18,1,1,20,0.044997,0.075171,3.023318,0.052067,2.505450
3,resnet18,2,1,1,0.064808,0.145800,3.759387,0.114669,0.134385
4,resnet18,2,1,10,0.074827,0.125240,1.836737,0.085478,1.278062
5,resnet18,2,1,20,0.086086,0.145713,7.982926,0.110346,2.592506
6,resnet18,8,1,1,0.056493,0.163512,1.902815,0.115756,0.129391
7,resnet18,8,1,10,0.152444,0.442749,4.467529,0.329146,1.312246
8,resnet18,8,1,20,0.155041,0.495774,4.248340,0.321058,2.568808
9,resnet18,32,1,1,0.065372,0.152724,1.899008,0.098565,0.128134
